### Impact schepcorner
In deze notebook gaan we na wat de impact is van de schepcorner in het minivoetbal. Hoe uitslagbepalend is hij. Leidt het missen van een schepcorner tot een negatieve sfeer. 

In [51]:
import pandas as pd

In [52]:
### data inladen
wedstrijden_cleaned = pd.read_excel('../../Data/Silver/wedstrijden_cleaned.xlsx')
corners_cleaned = pd.read_excel('../../Data/Silver/cornerballen_cleaned.xlsx')

In [53]:
merged_df = pd.merge(wedstrijden_cleaned, corners_cleaned, on='wedstrijd_id', how='left')

Aantal matchen zonder/met schepcorner

In [54]:
aantal_wedstrijden = wedstrijden_cleaned.shape[0]
aantal_wedstrijden_met_corners = corners_cleaned["wedstrijd_id"].nunique()


In [55]:
print(f"Aantal wedstrijden: {aantal_wedstrijden}")
print(f"Aantal wedstrijden met corners: {aantal_wedstrijden_met_corners}")
print(f"Aantal wedstrijden zonder corners: {aantal_wedstrijden - aantal_wedstrijden_met_corners}")
print(f"Percentage wedstrijden met corners: {aantal_wedstrijden_met_corners / aantal_wedstrijden * 100:.2f}%")

Aantal wedstrijden: 222
Aantal wedstrijden met corners: 221
Aantal wedstrijden zonder corners: 1
Percentage wedstrijden met corners: 99.55%


### strengste impact
veranderen van uitkomst van 1 schepcorner verandert resultaat van match, andere winnaar/gelijkspel

In [56]:
# strengste impact, uitkomst van 1 schepcorner in de match veranderen, is bepalend
def schepcorner_impact(df):
    results = []

    for match_id, match_data in df.groupby("wedstrijd_id"):
        uitslag_str = match_data["uitslag"].iloc[0]  # iloc selecteert eerste rij van match_data
        score_thuis, score_uit = map(int, uitslag_str.split("v"))  
        score_verschil = abs(score_thuis - score_uit)

        # Controleer of er een schepcorner is (indien alle is_thuisploeg en goal NA zijn, dan geen schepcorner)
        heeft_schepcorner = match_data["goal"].notna().any()

        if not heeft_schepcorner:
            results.append((match_id, False))  # Geen schepcorner? Geen impact.
            continue  

        # Gelijkspel: altijd impact als er een schepcorner was
        if score_thuis == score_uit:
            results.append((match_id, True))
            continue

        # Bepaal winnaar en verliezer
        if score_thuis > score_uit:
            winnaar = 1
            verliezer = 0
        else:
            winnaar = 0
            verliezer = 1

        # Controleer of winnaar minstens één schepcorner scoorde
        winnaar_heeft_gescoord = ((match_data["is_thuisploeg"] == winnaar) & (match_data["goal"] == 1)).any()

        # Controleer of verliezer minstens één schepcorner miste
        verliezer_heeft_gemist = ((match_data["is_thuisploeg"] == verliezer) & (match_data["goal"] == 0)).any()

        # Strikte voorwaarde: maximaal 1 verschil en schepcorner moet impact hebben
        if score_verschil == 1 and (winnaar_heeft_gescoord or verliezer_heeft_gemist):
            results.append((match_id, True))
        else:
            results.append((match_id, False))

    return pd.DataFrame(results, columns=["wedstrijd_id", "schepcorner_impact"])

In [57]:
strengste_impact_df = schepcorner_impact(merged_df)
aantal_matchen_met_strengste_impact = strengste_impact_df["schepcorner_impact"].sum()

In [58]:
print(f"Aantal wedstrijden met strengste impact: {aantal_matchen_met_strengste_impact}")
print(f"Percentage wedstrijden met strengste impact: {aantal_matchen_met_strengste_impact / aantal_wedstrijden * 100:.2f}%")

Aantal wedstrijden met strengste impact: 56
Percentage wedstrijden met strengste impact: 25.23%


In [59]:
# controle
wedstrijd_ids_strengste_impact = strengste_impact_df[strengste_impact_df["schepcorner_impact"] == 1]["wedstrijd_id"].tolist()
wedstrijd_ids_strengste_impact 

[6,
 12,
 16,
 18,
 26,
 29,
 30,
 34,
 37,
 40,
 42,
 44,
 45,
 46,
 49,
 50,
 62,
 66,
 68,
 71,
 84,
 88,
 93,
 100,
 111,
 114,
 115,
 116,
 121,
 126,
 127,
 147,
 150,
 152,
 153,
 156,
 165,
 166,
 167,
 174,
 176,
 177,
 181,
 183,
 185,
 187,
 190,
 193,
 198,
 200,
 201,
 205,
 208,
 209,
 216,
 219]

In [60]:
merged_df[merged_df["wedstrijd_id"].isin(wedstrijd_ids_strengste_impact)][["wedstrijd_id", "uitslag", "is_thuisploeg", "goal"]]

,wedstrijd_id,uitslag,is_thuisploeg,goal
13,6,7v8,1.0,1.0
14,6,7v8,1.0,1.0
15,6,7v8,0.0,1.0
29,12,5v6,1.0,1.0
30,12,5v6,0.0,1.0
...,...,...,...,...
553,216,5v4,0.0,1.0
554,216,5v4,1.0,1.0
561,219,5v4,1.0,1.0
562,219,5v4,0.0,1.0


### impact op matchuitslagen wanneer er geen schepcorners zouden zijn, of alle gemist zouden zijn.

In [61]:
def bepaal_winnaar_verliezer(uitslag_str):
    thuis_score, uit_score = map(lambda x: int(float(x)), uitslag_str.split('v'))
    
    if thuis_score > uit_score:
        return 1  # thuisploeg wint
    elif uit_score > thuis_score:
        return 0  # uitploeg wint
    else:
        return None  # gelijkspel


In [62]:
def bereken_impact_zonder_schepcorner(df):
    impact_lijst = []

    for wedstrijd_id, match_data in df.groupby("wedstrijd_id"):
        uitslag_str = match_data["uitslag"].iloc[0]
        
        # Bepaal het aantal schepcorner-goals per ploeg
        goals_thuis = match_data[match_data["is_thuisploeg"] == 1]["goal"].sum()
        goals_uit = match_data[match_data["is_thuisploeg"] == 0]["goal"].sum()

        # Bereken aangepaste score
        thuis_score, uit_score = map(int, uitslag_str.split('v'))
        nieuwe_thuis = thuis_score - goals_thuis
        nieuwe_uit = uit_score - goals_uit

        # Vergelijk oorspronkelijke en aangepaste uitslag
        oorspronkelijke = bepaal_winnaar_verliezer(uitslag_str)
        nieuwe = bepaal_winnaar_verliezer(f"{nieuwe_thuis}v{nieuwe_uit}")
        impact = oorspronkelijke != nieuwe

        impact_lijst.append({
            "wedstrijd_id": wedstrijd_id,
            "schepcorner_impact": impact
        })

    return pd.DataFrame(impact_lijst)


In [63]:
impact_zonder_schepcorners = bereken_impact_zonder_schepcorner(merged_df)
aantal_matchen_met_impact_zonder_schepcorner = impact_zonder_schepcorners["schepcorner_impact"].sum()

In [64]:
print(f"Aantal wedstrijden met andere uitslag moesten er geen schepcorners zijn: {aantal_matchen_met_impact_zonder_schepcorner}")
print(f"Percentage wedstrijden met andere uitslag moesten er geen schepcorners zijn: {aantal_matchen_met_impact_zonder_schepcorner / aantal_wedstrijden * 100:.2f}%")

Aantal wedstrijden met andere uitslag moesten er geen schepcorners zijn: 41
Percentage wedstrijden met andere uitslag moesten er geen schepcorners zijn: 18.47%


### impact alle schepcorners gescoord


In [65]:
def bereken_schepcorner_impact_als_alle_gescoord(df):
    impact_lijst = []

    for wedstrijd_id, match_data in df.groupby("wedstrijd_id"):
        uitslag_str = match_data["uitslag"].iloc[0]

        # Aantal gemiste schepcorners per ploeg
        gemist_thuis = ((match_data["is_thuisploeg"] == 1) & (match_data["goal"] == 0)).sum()
        gemist_uit = ((match_data["is_thuisploeg"] == 0) & (match_data["goal"] == 0)).sum()

        # Score aanpassen alsof elke schepcorner werd gescoord
        thuis_score, uit_score = map(int, uitslag_str.split('v'))
        nieuwe_thuis = thuis_score + gemist_thuis
        nieuwe_uit = uit_score + gemist_uit

        # Bepaal winnaar voor en na
        oorspronkelijke = bepaal_winnaar_verliezer(f"{thuis_score}v{uit_score}")
        nieuwe = bepaal_winnaar_verliezer(f"{nieuwe_thuis}v{nieuwe_uit}")
        impact = oorspronkelijke != nieuwe

        impact_lijst.append({
            "wedstrijd_id": wedstrijd_id,
            "schepcorner_impact": impact
        })

    return pd.DataFrame(impact_lijst)


In [66]:
impact_alles_gescoord = bereken_schepcorner_impact_als_alle_gescoord(merged_df)
aantal_matchen_met_impact_alles_gescoord = impact_alles_gescoord["schepcorner_impact"].sum()

In [67]:
print(f"Aantal wedstrijden met andere uitslag moesten alle schepcorners gescoord worden: {aantal_matchen_met_strengste_impact}")
print(f"Percentage wedstrijden met andere uitslag moesten alle schepcorners gescoord worden: {aantal_matchen_met_impact_alles_gescoord / aantal_wedstrijden * 100:.2f}%")

Aantal wedstrijden met andere uitslag moesten alle schepcorners gescoord worden: 56
Percentage wedstrijden met andere uitslag moesten alle schepcorners gescoord worden: 12.16%


### schepcorner impact, minst streng berekend
Hoeveel wedstrijden hebben een andere uitslag, als alle schepcorners van uitkomst kunnen veranderen

In [68]:
def bereken_schepcorner_impact_mild(df):
    impact_lijst = []

    for wedstrijd_id, groep in df.groupby("wedstrijd_id"):
        uitslag_str = groep["uitslag"].iloc[0]

        # Scores omzetten naar integers (voor veiligheid float -> int)
        thuis_score, uit_score = map(int, uitslag_str.split('v'))
        oorspronkelijke = bepaal_winnaar_verliezer(f"{thuis_score}v{uit_score}")

        # Tel gescoorde en gemiste corners
        gescoord_thuis = ((groep["is_thuisploeg"] == 1) & (groep["goal"] == 1)).sum()
        gemist_thuis = ((groep["is_thuisploeg"] == 1) & (groep["goal"] == 0)).sum()
        gescoord_uit = ((groep["is_thuisploeg"] == 0) & (groep["goal"] == 1)).sum()
        gemist_uit = ((groep["is_thuisploeg"] == 0) & (groep["goal"] == 0)).sum()

        # Als gelijkspel en er waren corners → impact
        if oorspronkelijke is None and (gescoord_thuis + gemist_thuis + gescoord_uit + gemist_uit) > 0:
            impact = True
        else:
            if oorspronkelijke == 1:
                min_win = thuis_score - gescoord_thuis
                max_verl = uit_score + gemist_uit
            else:
                min_win = uit_score - gescoord_uit
                max_verl = thuis_score + gemist_thuis

            nieuwe = bepaal_winnaar_verliezer(f"{min_win}v{max_verl}")
            impact = oorspronkelijke != nieuwe

        impact_lijst.append({
            "wedstrijd_id": wedstrijd_id,
            "schepcorner_impact": impact
        })

    return pd.DataFrame(impact_lijst)


In [69]:
impact_mild = bereken_schepcorner_impact_mild(merged_df)
aantal_matchen_met_impact_mild = impact_mild["schepcorner_impact"].sum()

In [70]:
print(f"Aantal wedstrijden met andere uitslag moesten alle schepcorners van uitkomst gewijzigd kunnen worden: {aantal_matchen_met_impact_mild}")
print(f"Percentage wedstrijden met andere uitslag moesten alle schepcorners van uitkomst gewijzigd kunnen worden: {aantal_matchen_met_impact_mild / aantal_wedstrijden * 100:.2f}%")

Aantal wedstrijden met andere uitslag moesten alle schepcorners van uitkomst gewijzigd kunnen worden: 131
Percentage wedstrijden met andere uitslag moesten alle schepcorners van uitkomst gewijzigd kunnen worden: 59.01%


### Nu kunnen we ook checken of corners missen/scoren sleutelmomenten zijn

In [72]:
merged_df.columns

Index(['wedstrijd_id', 'datum', 'niveau', 'uitslag', 'corners_uitslag',
       'klassement_uitslag', 'schepcorner_id', 'ploegnaam',
       'ervaring_schepper', 'ervaring_kopper', 'ervaring_duo', 'is_thuisploeg',
       'kwart', 'is_schepper_bank', 'is_kopper_bank', 'score_voor',
       'schepper_pos_rechts', 'positie', 'kop_verplaatsen_achter',
       'kop_veel_verplaatsen', 'kop_hor_verplaatsen', 'goal'],
      dtype='object')

In [74]:
merged_df[["wedstrijd_id", "uitslag", "schepcorner_id", "is_thuisploeg", "score_voor", "goal"]].head()

,wedstrijd_id,uitslag,schepcorner_id,is_thuisploeg,score_voor,goal
0,1,4v1,1.0,1.0,0v0,1.0
1,1,4v1,2.0,1.0,2v0,0.0
2,1,4v1,3.0,0.0,2v0,0.0
3,2,11v2,4.0,1.0,4v1,0.0
4,2,11v2,5.0,0.0,5v2,0.0


### hypothese, missen schepcorner leidt tot momentum_verlies. 

In [76]:
def bepaal_sleutelmomenten_schepcorners(df):
    sleutelmomenten = []

    for wedstrijd_id, match_data in df.groupby("wedstrijd_id"):
        match_data = match_data.sort_values("schepcorner_id").reset_index(drop=True)

        # Eindscore parsen
        eind_thuis, eind_uit = map(int, match_data["uitslag"].iloc[0].split('v'))

        for i, rij in match_data.iterrows():
            if rij["goal"] != 0:
                continue  # Alleen gemiste schepcorners analyseren

            # Score vóór de corner
            thuis_voor, uit_voor = map(int, rij["score_voor"].split('v'))
            verschil_voor = thuis_voor - uit_voor

            # Score ná de corner: volgende snapshot of eindstand
            if i + 1 < len(match_data):
                thuis_na, uit_na = map(int, match_data.loc[i + 1, "score_voor"].split('v'))
            else:
                thuis_na, uit_na = eind_thuis, eind_uit
            verschil_na = thuis_na - uit_na

            delta = verschil_voor - verschil_na

            # Impact bepalen volgens ploeg
            if rij["is_thuisploeg"] == 1:
                sleutelmoment = delta > 0
            else:
                sleutelmoment = delta < 0

            sleutelmomenten.append({
                "wedstrijd_id": wedstrijd_id,
                "schepcorner_id": rij["schepcorner_id"],
                "sleutelmoment": sleutelmoment
            })

    return pd.DataFrame(sleutelmomenten)


In [77]:
impact_missen = bepaal_sleutelmomenten_schepcorners(merged_df)
aantal_sleutelmomenten = impact_missen["sleutelmoment"].sum()

In [81]:
print(f"Aantal schepcorners gemist: {merged_df[merged_df['goal'] == 0].shape[0]}")
print(f"Aantal gemiste schepcorners die leiden tot momentum verlies: {aantal_sleutelmomenten}")
print(f"Percentage gemiste schepcorners die leiden tot momentum verlies: {aantal_sleutelmomenten / merged_df[merged_df['goal'] == 0].shape[0] * 100:.2f}%")

Aantal schepcorners gemist: 201
Aantal gemiste schepcorners die leiden tot momentum verlies: 52
Percentage gemiste schepcorners die leiden tot momentum verlies: 25.87%


### positieve impact na gescoorde corner.

In [83]:
def bepaal_sleutelmomenten_gescoorde_schepcorners(df):
    sleutelmomenten = []

    for wedstrijd_id, groep in df.groupby("wedstrijd_id"):
        groep = groep.sort_values("schepcorner_id").reset_index(drop=True)

        # Eindscore parsen
        eind_thuis, eind_uit = map(int, groep["uitslag"].iloc[0].split('v'))

        for i, rij in groep.iterrows():
            if rij["goal"] != 1:
                continue  # Alleen gescoorde schepcorners analyseren

            # Score vóór de corner, goal direct bijtellen
            thuis_voor, uit_voor = map(int, rij["score_voor"].split('v'))
            if rij["is_thuisploeg"] == 1:
                thuis_voor += 1
            else:
                uit_voor += 1
            verschil_voor = thuis_voor - uit_voor

            # Score ná de corner: volgende snapshot of eindstand
            if i + 1 < len(groep):
                thuis_na, uit_na = map(int, groep.loc[i + 1, "score_voor"].split('v'))
            else:
                thuis_na, uit_na = eind_thuis, eind_uit
            verschil_na = thuis_na - uit_na

            delta = verschil_na - verschil_voor

            # Impact bepalen volgens ploeg
            if rij["is_thuisploeg"] == 1:
                sleutelmoment = delta > 0
            else:
                sleutelmoment = delta < 0

            sleutelmomenten.append({
                "wedstrijd_id": wedstrijd_id,
                "schepcorner_id": rij["schepcorner_id"],
                "sleutelmoment": sleutelmoment
            })

    return pd.DataFrame(sleutelmomenten)


In [84]:
impact_scoren = bepaal_sleutelmomenten_gescoorde_schepcorners(merged_df)
aantal_sleutelmomenten_score = impact_scoren["sleutelmoment"].sum()

In [85]:
print(f"Aantal schepcorners gemist: {merged_df[merged_df['goal'] == 1].shape[0]}")
print(f"Aantal gescoorde schepcorners die leiden tot momentum boost: {aantal_sleutelmomenten}")
print(f"Percentage gescoorde schepcorners die leiden tot momentum boost: {aantal_sleutelmomenten / merged_df[merged_df['goal'] == 1].shape[0] * 100:.2f}%")

Aantal schepcorners gemist: 370
Aantal gescoorde schepcorners die leiden tot momentum boost: 52
Percentage gescoorde schepcorners die leiden tot momentum boost: 14.05%


Heel lage percentages, kan zijn omdat schepcorners vaak dicht op elkaar volgen en daardoor er nog niet echt tijd is voor een groot verschil van boost